In [71]:
# import libraries and modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import gmaps
import scipy.stats as scistats
import random
import requests
import json
from datetime import datetime
from config import api_key

In [78]:
# Kristina: Module to generate list of movie IDs

In [79]:
# mock data
dataDF = pd.DataFrame({'1970':['10671','11202','10112','10671','11202','10112'],
                       '1980':['651','11165','10364','651','11165','10364'],
                       '1990':['1685','64512','64512','10112', '9062', '11165'],
                      '2000':['1685','64512','64512','10112', '9062', '11165'],
                      '2010':['1685','64512','64512','10112', '9062', '11164']})

# Evan: Module to get list of people from cast
def top2people(movies):
    
    peopleDict = {'1970':[],'1980':[],'1990':[],'2000':[],'2010':[]}

    # loop through each movie in each column
    for column in movies.iteritems():
        for movie in column[1]:

            # request credits list for movie
            cast_url = f'https://api.themoviedb.org/3/movie/{movie}/credits?api_key={api_key}'
            data = requests.get(cast_url).json()

            # get list of cast IDs
            idList = {}
            for n,x in enumerate(data['cast']):
                idList[n] = x['cast_id']

            # determine top 2 IDs
            idDF = pd.DataFrame.from_dict(idList, orient='index', columns=['cast_id'])
            idDF.sort_values('cast_id', inplace=True)
            idDF.reset_index(drop=False, inplace=True)

            # get person ID for top 2
            person1 = data['cast'][idDF.iloc[0]['index']]['id']
            person2 = data['cast'][idDF.iloc[1]['index']]['id']

            # check for uniqueness 
            if person1 not in peopleDict[column[0]]:
                peopleDict[column[0]].append(person1)
            if person2 not in peopleDict[column[0]]:
                peopleDict[column[0]].append(person2)
                
    # convert dictionary of people IDs to DataFrame
    peopleDF = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in peopleDict.items()]))
    return peopleDF

In [81]:
mockpeopleIDs = top2people(dataDF)

In [10]:
# Aruna: Module to generate DataFrame of people with birth state and places

In [ ]:
# Module to plot birthplaces on gmaps